In [19]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [20]:
# Ouverture du fichier csv
chemin_fichier_csv = r'./champignons.csv'
champignons = pd.read_csv(chemin_fichier_csv)

print(champignons.head())

  TYPE         COLOR       SHAPE     SURFACE
0    I          Pale    Polypore      Smooth
1    E         White      Convex      Smooth
2    E  White-Yellow      Convex      Smooth
3    E         Brown  Bellshaped  FlatScales
4    E           NaN      Convex     Fibrous


In [21]:
# Modification des valeur de la collone TYPE
print(champignons["TYPE"].value_counts(dropna=False))

champignons["TYPE"] = champignons["TYPE"].replace({"E": 0, "I": 1, "P": 2})
champignons["TYPE"] = champignons["TYPE"].fillna(-1)

print(champignons.head())
print(champignons["TYPE"].value_counts(dropna=False))

TYPE
E      515
P      405
I      228
NaN     36
Name: count, dtype: int64
   TYPE         COLOR       SHAPE     SURFACE
0   1.0          Pale    Polypore      Smooth
1   0.0         White      Convex      Smooth
2   0.0  White-Yellow      Convex      Smooth
3   0.0         Brown  Bellshaped  FlatScales
4   0.0           NaN      Convex     Fibrous
TYPE
 0.0    515
 2.0    405
 1.0    228
-1.0     36
Name: count, dtype: int64


In [22]:
# Tranformation des colonnes catégorielles(SHAPE/SURFACE) en colonnes binaires
def ajout_indicateur_colonne(df, nom_colonne):
    valeur_unique = pd.unique(df[nom_colonne].str.split("-").explode().dropna())
    
    for value in valeur_unique:
        df[f"{nom_colonne}_{value}"] = df[nom_colonne].str.contains(value, na=False).astype(int)

ajout_indicateur_colonne(champignons, "SHAPE")
ajout_indicateur_colonne(champignons, "SURFACE")

champignons = champignons.drop(['SHAPE', 'SURFACE'], axis=1)

print(champignons.head())

   TYPE         COLOR  SHAPE_Polypore  SHAPE_Convex  SHAPE_Bellshaped  \
0   1.0          Pale               1             0                 0   
1   0.0         White               0             1                 0   
2   0.0  White-Yellow               0             1                 0   
3   0.0         Brown               0             0                 1   
4   0.0           NaN               0             1                 0   

   SHAPE_Depressed  SHAPE_CupFungi  SHAPE_CoralFungi  SHAPE_Conical  \
0                0               0                 0              0   
1                0               0                 0              0   
2                0               0                 0              0   
3                0               0                 0              0   
4                0               0                 0              0   

   SHAPE_SiddarthMachado  ...  SURFACE_Velvety  SURFACE_AlanRockefeller  \
0                      0  ...                0             